In [14]:
import Data as Data
from keras.layers import Reshape, merge, Input, Dense, Flatten
from keras.engine import  Model
from keras_vggface import VGGFace
from scipy.spatial.distance import cosine
import numpy as np
import scipy.misc
import PIL
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.decomposition import PCA

In [10]:
def data_init():
    print('Data loading...')
    (X_1,X_2,Y) = Data.loadData('/home/danning/lfw/lfw_data.h5')
     #len = 6000
    
        
    print('Image resizing...')
    Xr_1 = []
    Xr_2 = []
    for i in range(len(X_1)):
        Xr_1.append(scipy.misc.imresize(X_1[i], size=(224,224,3),interp='nearest'))
        Xr_2.append(scipy.misc.imresize(X_2[i], size=(224,224,3),interp='nearest'))
    
    print('Image flipping...')

    Xf_1 = [] #image array after flipping
    Xf_2 = []
    
    for i in range(len(X_1)):
        img1 = PIL.Image.fromarray(Xr_1[i], mode=None)
        img2 = PIL.Image.fromarray(Xr_2[i], mode=None)
        Xf_1.append(np.array(img1.transpose(Image.FLIP_LEFT_RIGHT)))
        Xf_2.append(np.array(img2.transpose(Image.FLIP_LEFT_RIGHT)))

    Xr_1 = np.asarray(Xr_1)
    Xr_2 = np.asarray(Xr_2)
    Xf_1 = np.asarray(Xf_1)
    Xf_2 = np.asarray(Xf_2) 
    
    print('Image normalizing...')
    Xr_1 = (Xr_1.astype(np.float32) - 127.5)/127.5
    Xr_2 = (Xr_2.astype(np.float32) - 127.5)/127.5
    Xf_1 = (Xf_1.astype(np.float32) - 127.5)/127.5
    Xf_2 = (Xf_2.astype(np.float32) - 127.5)/127.5


    
    
    return (Xr_1,Xr_2,Xf_1,Xf_2,Y)
        

In [3]:
'''
(Xr_1,Xr_2,Xf_1,Xf_2,Y) = data_init()
plt.subplot(121)  
plt.imshow(Xr_1[0])  
plt.subplot(122)  
plt.imshow(Xf_1[0])
'''

'\n(Xr_1,Xr_2,Xf_1,Xf_2,Y) = data_init()\nplt.subplot(121)  \nplt.imshow(Xr_1[0])  \nplt.subplot(122)  \nplt.imshow(Xf_1[0])\n'

In [4]:
def vgg_model():
    vgg_model = VGGFace(include_top=True, input_shape=(224, 224, 3), pooling='avg') # pooling: None, avg or max
    out = vgg_model.get_layer('fc7').output
    vgg_model = Model(vgg_model.input, out)
    vgg_model.summary()
    return vgg_model

In [41]:
def verification(model,data,pca = False):
    (Xr_1,Xr_2,Xf_1,Xf_2,Y) = data
    
    print('Feature vectors extracting...')
    fvec11=model.predict(Xr_1, verbose=0)
    fvec12=model.predict(Xf_1, verbose=0)
    fvec21=model.predict(Xr_2, verbose=0)
    fvec22=model.predict(Xf_2, verbose=0)
    
    fvec1 = fvec11 + fvec12
    fvec2 = fvec21 + fvec22
    
    
    
    if pca:
        print('Linear dimensionality reduction...')
        '''
        sklearn_pca = PCA(n_components=128,svd_solver='arpack')
        fvec1 = sklearn_pca.fit_transform(fvec1)
        fvec2 = sklearn_pca.fit_transform(fvec2)
        '''
        #waiting to be completed
        
    print('Cosine similarity calculating...')
    cos_sim = []
    for i in range(len(fvec1)):
        cos_sim.append(1-cosine(fvec1[i],fvec2[i]))
    #print(cos_sim)

    print('cos_sim range: {0},{1}'.format(min(cos_sim),max(cos_sim)))
    
    
    f = [] #f_measurement
    max_f = 0 
    max_f_index = -1 
    pre = [] #precision
    rec = [] #recall
    for i in range(len(cos_sim)):
        threshold = cos_sim[i]
        tp=0 #true positive
        fp=0 #false positive
        fn=0 #false negative
        for j in range(len(cos_sim)):
            if cos_sim[j]>=threshold:
                if Y[j] == 1:
                    tp+=1
                if Y[j] == 0:
                    fp+=1
            else:
                if Y[j] == 1:
                    fn+=1
        pre.append(tp/(tp+fp))
        rec.append(tp/(tp+fn))
        temp = 2*pre[i]*rec[i]/(pre[i]+rec[i])
        f.append(temp)
        if temp>max_f:
            max_f = temp
            max_f_index = i

    threshold = cos_sim[max_f_index]
    print('threshold:{0}'.format(threshold))
    
    
    y_test = []
    err=0
    for i in range(len(cos_sim)):
        y_test.append(1 if cos_sim[i]>=threshold else 0) 
        if Y[i]!=y_test[i]:
            err+=1

    acc=1-err/len(Xr_1)
    print(acc)

In [13]:
data = data_init()
model = vgg_model()
verification(model, data)

Data loading...
Image resizing...
Image flipping...
Image normalizing...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_1 (Conv2D)             (None, 224, 224, 64)      1792      
_________________________________________________________________
conv1_2 (Conv2D)             (None, 224, 224, 64)      36928     
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 112, 112, 64)      0         
_________________________________________________________________
conv2_1 (Conv2D)             (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2_2 (Conv2D)             (None, 112, 112, 128)     147584    
_________________________________________________________________
poo